In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c
import seaborn as sns

In [3]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('Customized_Components')
from rounded_single_pad import Round_TransmonPocket_Single as transmon
import Transmon_property as trans_p
import Transmon_specifications as jj

import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [4]:
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [5]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

In [6]:
import seaborn as sns

In [7]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45
cc = 1664080*u.fF/u.m

In [8]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '20 mm'
design.chips.main.size['size_y'] = '10 mm'

design.variables['cpw_width'] = '15.9 um'
design.variables['cpw_gap'] = '9 um'

design.variables['trace_width'] = '12.4 um'
cpw_pin_width = 12.4*u.um
design.variables['trace_gap'] = '7 um'
cpw_gap = 7*u.um

design.variables['pad_buffer_radius'] = '30 um'
design.variables['buffer_resolution'] = '10'
design.variables['connection_pad_buffer_radius'] = '2 um'

# design.chips['main']['material'] = 'Silicon'

gui = MetalGUI(design)

05:51PM 40s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Populating font family aliases took 149 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 



Component-specific variables

In [9]:


qb_options = dict(
pad_pocket_distance_top = '40um',
connection_pads = dict(
    a = dict(loc_W  = 0, 
            loc_H =  1, 
            pad_gap = '10um',
            pad_height = '20um',
            pad_width = '80um',
            pad_cpw_extent =  '10um',
            pocket_rise = '0um',
            cpw_extend = '5um',
            pocket_extent = '1um')))


TQ_options = dict(prime_width = design.variables['cpw_width'],
               prime_gap = design.variables['cpw_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '60um',
               coupling_space = '5um',
               open_termination=False,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False)

CPW_options = Dict(trace_width = design.variables['trace_width'],
               trace_gap  = design.variables['trace_gap'],
        total_length='5 mm',
        hfss_wire_bonds = True,
        q3d_wire_bonds = True,
        fillet='30 um',
        lead = dict(start_straight='20um', end_straight = '50um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end')), )

pin_inputs = Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end'))

trans_options = Dict(hfss_wire_bonds = True,
                     q3d_wirebonds = True,
               pin_inputs=Dict(
                 start_pin=Dict(
                     component='TQ1',
                     pin='prime_start'),
                 end_pin=Dict(
                     component='TQ2',
                     pin='prime_end')))


## Single Pad--Verify Design

### Draw the single pad transmon

In [ ]:
design.delete_all_components()

In [ ]:
design.overwrite_enabled = True

size = 0.25*u.mm
gap = 30*u.um
size = size.to(u.um)
pocket_width = size+2*gap


op = Dict(
    pos_x = '0',
    pad_height = '{}um'.format(size.value),
    pos_y = '0',
    pad_width = '{}'.format(size),
    pocket_width = '{}'.format(pocket_width), 
    **qb_options)

q1 = transmon(design,'Q1',options = op)

gui.rebuild()
gui.zoom_on_components(['Q1'])


In [12]:
design.parse_value('13')

13.0

In [14]:
from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.analyses.quantization import EPRanalysis


In [15]:
renderer_hfss = design.renderers.hfss
renderer_q3d = design.renderers.q3d

In [ ]:
c1 = LOManalysis(design, "q3d")
c1.sim.setup.min_passes  = 10
c1.sim.setup.max_passes = 25
c1.sim.setup.freq_ghz = 5

c1.sim.renderer.options['wb_threshold'] ='72um'
c1.sim.renderer.options['x_buffer_width_mm'] = 0.5
c1.sim.renderer.options['y_buffer_width_mm'] = 0.5

In [16]:
eig_all = EPRanalysis(design, 'hfss')

hfss = eig_all.sim.renderer

# eig_all.sim.renderer.options['Lj'] = '13nH'
eig_all.sim.renderer.options['wb_threshold'] = '72um'

eig_all.sim.setup.max_passes = 30
eig_all.sim.setup.max_delta_f = 0.1
eig_all.sim.setup.n_modes = 1
# eig_all.sim.setup.vars = Dict(Lj1= '13 nH', Cj1= '0 fF')
eig_all.sim.renderer.options['x_buffer_width_mm'] = 0.5
eig_all.sim.renderer.options['y_buffer_width_mm'] = 0.5

In [ ]:
data = pd.DataFrame()

In [13]:
int('13nH'[:-2])

13

In [ ]:
original = r'C:\Users\slab\Desktop\Wendy-qiskit-code\ansys.png'
path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation_0401'

In [ ]:
nmode = 1
for Lj in ['13nH','10nH']:
    Cj = jj.find_junction_capacitance(int(Lj[:-2])*u.fF)
    for size in np.linspace(250,170,5)*u.um:
        dat = {}
        dat['Lj (nH)'] = int(Lj[:-2])
        dat['size (um)'] = size
        dat['Cj (fF)'] = Cj.to(u.fF).value
        Cj = string(Cj.to(u.fF).value)+' fF'

        eig_all.sim.renderer.options['Lj'] = Lj
        eig_all.sim.renderer.options['Cj'] = Cj

        eig_all.sim.setup.vars = Dict(Lj1= Lj, Cj1= Cj)

        # gap1 = 0.055
        gap = 30*u.um
        size = pad_size.to(u.um)
        pocket_width = size+2*gap

        # coupling_len = extend
        q1.options['pad_height'] = '{}'.format(size)
        q1.options['pad_width'] = '{}'.format(size)
        q1.options['pocket_width'] = '{}'.format(pocket_width)

        c1.sim.run(components=['Q1'])#, open_terminations=[('Q1', 'a')])#, ('Q1', 'bus1'), ('Q1', 'bus2')])
        c1.sim.capacitance_matrix 

        c1.sim._get_results_from_renderer()
        c_mat1 = c1.sim.capacitance_matrix
        renderer_q3d.clean_active_design()
        dat['C_CPW_Qb'] = c_mat1


        c1.sim.run(components=['Q1'], open_terminations=[('Q1', 'a')])#, ('Q1', 'bus1'), ('Q1', 'bus2')])
        c1.sim.capacitance_matrix 

        c1.sim._get_results_from_renderer()
        c_mat1 = c1.sim.capacitance_matrix
        renderer_q3d.clean_active_design()
        dat['C_CPW_Qb_g'] = c_mat1


        eig_all.sim.run(name="all_freqs", components=['Q1'])#, open_terminations=[('Q1', 'a')])
        convergence = pd.read_csv('hfss_eig_f_convergence.csv')
        conv = convergence.dropna()

        
        ind = list(conv.keys())[1:]
        freq = conv[ind[0]].values[-1]
        dat['Freq_conv(GHz)'] = freq

        for i in range(nmode):
                hfss.modeler._modeler.ShowWindow()
                hfss.set_mode(i+1,'Setup')
                hfss.plot_ansys_fields('main')
                hfss.save_screenshot()
                hfss.clear_fields(['main'])
                
                actual = r'C:\Users\slab\Desktop\Wendy-qiskit-code\size{:.3f}'.format((size.value))+Lj+'.png'
                os.rename(original, actual)
                fail = True
                i = 0
                while fail:

                    try:
                        dest = shutil.move(actual, path)
                    except:
                        i+= 1
                        os.rename(actual,(actual+'{}'.format(i)))
                        actual = (actual+'{}'.format(i))
                    else:
                        fail = False


        try:
            del eig_all.setup.junctions['jj']
        except:
            print('saddd')
        eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                        Lj_variable='Lj1', Cj_variable='Cj1')
        eig_all.setup.sweep_variable = 'Lj1'
        eig_all.run_epr()

        #Save the EPR data
        chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
        freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]
        
        renderer_hfss.clean_active_design()
        for i in range(nmode):
            freq = freq_EPR[i]
            dat['Freq_EPR'+str(i+1)+'(MHz)'] = freq
            for j in range(i+1):
                dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j]

        data = pd.DataFrame(dat, index = [0])
        datas = pd.concat([datas,data], ignore_index=True)


        datas.to_csv('data\QB_freq_only_0401.csv')





        